In [1]:
!pip install pytorch-lightning ray[tune]

In [2]:
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.9 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.9
    Uninstalling widgetsnbextension-3.6.9:
      Successfully uninstalled widgetsnbextension-3.6.9
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.11
    Uninstalling jupyterlab_widgets-3.0.11:
      Successfully uninstalled jupyterlab_widgets-3.0.11
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0

In [3]:
import os
FILE_NAMES = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        FILE_NAMES.append(os.path.join(dirname, filename))

In [4]:
# print(FILE_NAMES)

In [5]:
# import torch
# torch.cuda.is_available()

In [6]:
# !nvidia-smi

In [7]:
import os
from pathlib import Path
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import LearningRateMonitor, EarlyStopping
from ray.tune.integration.pytorch_lightning import TuneReportCheckpointCallback
from ray import train, tune
from ray.tune import Tuner, with_resources
# from ray_lightning.tune import TuneReportCallback
from ray.train import RunConfig

In [8]:
# Set global parameters
scale_factor = 11.888623072966611
input_dim = 3072
# sample_rate = 0.1 # 10% of data
sample_size = 8192 # file_size * sample_rate
num_epochs = 5  # Only one epoch over all data

# Dataset with Subsampling
class ActivationDataset(Dataset):
    def __init__(self, f_type, test_fraction=0.01, scale_factor=1.0, batch_size=2048, seed=42):
        self.test_fraction = test_fraction
        self.scale_factor = scale_factor
        self.batch_size = batch_size
        self.multi = sample_size // batch_size
        self.seed = seed
        self.file_names = FILE_NAMES
        if f_type not in ["train", "test"]:
            raise ValueError("f_type must be 'train' or 'test'")
        if f_type == "train":
            self.file_names = self.file_names[:int(len(self.file_names)*(1 - self.test_fraction))]
        else:
            self.file_names = self.file_names[int(len(self.file_names)*(1 - self.test_fraction)):]
        self.f_type = f_type

    def __len__(self):
        return len(self.file_names)*self.multi

    def __getitem__(self, idx):
        # Load a single file
        f_ix = idx // self.multi # sample the file 4 times
        file_path = self.file_names[f_ix]
        activations = np.load(file_path)[:, :-3]  # Remove metadata columns

        # Normalize
        activations = activations / self.scale_factor * np.sqrt(activations.shape[1])

        # Random subsampling to sample_size
        np.random.seed(self.seed + idx)  # Change seed per file for reproducibility
        subsample_indices = np.random.choice(activations.shape[0], sample_size, replace=False)
        activations = activations[subsample_indices]

        # Get batch 
        batch_i = idx % self.multi
        start = batch_i*self.batch_size
        end = (batch_i+1)*self.batch_size
        activations = activations[start:end]

        # Convert to tensor
        return torch.tensor(activations, dtype=torch.float32)

# Model Definition
class SparseAutoencoder(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, l1_lambda, lr):
        super().__init__()
        self.save_hyperparameters()
        self.encoder = nn.Linear(input_dim, hidden_dim)
        self.decoder = nn.Linear(hidden_dim, input_dim)
        self.criterion = nn.MSELoss()
        self.l1_lambda = l1_lambda
        self.lr = lr

    def forward(self, x):
        encoded = torch.relu(self.encoder(x))
        decoded = self.decoder(encoded)
        return decoded, encoded

    def compute_loss(self, batch, decoded, encoded):
        mse_loss = self.criterion(decoded, batch)
        decoder_weight_norms = torch.norm(self.decoder.weight, p=2, dim=0)
        l1_terms = encoded * decoder_weight_norms.unsqueeze(0)
        l1_loss = torch.mean(torch.sum(l1_terms, dim=1))
        return mse_loss, l1_loss

    def training_step(self, batch, batch_idx):
        batch = batch.to("cuda")
        decoded, encoded = self(batch)
        mse_loss, l1_loss = self.compute_loss(batch, decoded, encoded)
        total_loss = mse_loss + self.l1_lambda * l1_loss

        # Compute active features
        active_features = (encoded > 0).any(dim=0).float().mean().item() * 100

        # Log metrics
        self.log("train_loss", total_loss, on_step=True, on_epoch=True)
        self.log("train_mse_loss", mse_loss, on_step=True, on_epoch=True)
        self.log("train_l1_loss", l1_loss, on_step=True, on_epoch=True)
        self.log("active_features", active_features, on_step=True, on_epoch=True)
        self.log("val_loss", 0, on_step=True, on_epoch=True)
        # train.report({"loss": total_loss, "active_features": active_features})
        return total_loss

    def validation_step(self, batch, batch_idx):
        batch = batch.to("cuda")
        decoded, encoded = self(batch)
        mse_loss, l1_loss = self.compute_loss(batch, decoded, encoded)
        total_loss = mse_loss + self.l1_lambda * l1_loss

        # Compute active features
        active_features = (encoded > 0).any(dim=0).float().mean().item() * 100

        # Log metrics
        self.log("val_loss", total_loss, on_step=False, on_epoch=True)
        self.log("val_mse_loss", mse_loss, on_step=False, on_epoch=True)
        self.log("val_l1_loss", l1_loss, on_step=False, on_epoch=True)
        self.log("val_active_features", active_features, on_step=False, on_epoch=True)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
        return {"optimizer": optimizer, "lr_scheduler": scheduler}

# DataLoader Creation
def create_data_loaders(batch_size):
    train_dataset = ActivationDataset("train", 0.01, scale_factor, batch_size, 42)
    val_dataset = ActivationDataset("test", 0.01, scale_factor, batch_size, 42)
    # train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)#, num_workers=3, pin_memory=True, persistent_workers=True)
    # val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
    train_loader = DataLoader(
        train_dataset,
        batch_size=1,  # Keep 1 as outer batch size for per-file sampling
        shuffle=False,
        num_workers=3,  # Adjust based on CPU availability
        pin_memory=False,
        persistent_workers=True,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=1,  # Smaller for validation
        pin_memory=False,
    )
    return train_loader, val_loader

# Training Function with Ray Tune
def train_model(config):
    train_loader, val_loader = create_data_loaders(config["HB"]["batch_size"])
    model = SparseAutoencoder(input_dim, hidden_dim=config["HB"]["hidden_dim"], l1_lambda=config["l1_lambda"], lr=config["lr"])

    logger = TensorBoardLogger("tb_logs", name="SparseAutoencoder")

    trainer = pl.Trainer(
        max_epochs=num_epochs,
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        logger=logger,
        profiler="advanced",
        # val_check_interval=0.25,  # Check validation 4 times per epoch
        # max_time="00:30:00",  # Stop after 30 minutes
        enable_progress_bar=True, # Show progress bar
        callbacks=[
            LearningRateMonitor(logging_interval="step"),
            EarlyStopping(monitor="val_loss", patience=3, mode="min"),
            TuneReportCheckpointCallback(
                {
                    "train_loss": "train_loss",
                    "train_mse_loss": "train_mse_loss",
                    "train_l1_loss": "train_l1_loss",
                    "active_features": "active_features",
                    "val_loss": "val_loss",
                },
                filename="none",  # Do not save checkpoints
                save_checkpoints = False,
                on="train_batch_end",
            ),
            TuneReportCheckpointCallback(
                {
                    "val_loss": "val_loss",
                    "val_mse_loss": "val_mse_loss",
                    "val_l1_loss": "val_l1_loss",
                    "val_active_features": "val_active_features",
                },
                on="validation_end",
            ),
            # RayTrainReportCallback(),
        ],
        # strategy=RayDDPStrategy(), # Use Ray for distributed training, DDP stands for Distributed Data Parallel
        # callbacks=[RayTrainReportCallback()], # Report metrics to Ray
        # plugins=[RayLightningEnvironment()], # Use Ray for distributed training
    )
    trainer.fit(model, train_loader, val_loader)

# Ray Tune Hyperparameter Search
def tune_hyperparameters():

    # possible_hidden_dims = [4096, 8192, 16384, 20000, 32768]
    # possible_batch_sizes = [512, 1024, 2048, 4096, 8192]
    # possible_hidden_dims = [8192, 16384, 20000, 32768]
    # possible_batch_sizes = [1024, 2048, 4096, 8192]
    possible_hidden_dims = [65536]
    possible_batch_sizes = [2048]

    valid_hb_pairs = []
    for hidden_dim in possible_hidden_dims:
        for batch_size in possible_batch_sizes:
            if hidden_dim * batch_size <= 441_000_000: # VRAM limit
                valid_hb_pairs.append({"hidden_dim": hidden_dim, "batch_size": batch_size})


    # search_space = {
    #     "hidden_dim": tune.choice([4096, 8192, 16384, 20000, 32768]),
    #     "batch_size": tune.choice([512, 1024, 2048, 4096, 8192]),
    #     "l1_lambda": tune.loguniform(1e-4, 1e-2),
    #     "lr": tune.loguniform(1e-4, 1e-2),
    # }

    # search_space = {
    #     "HB": tune.choice(valid_hb_pairs),
    #     "l1_lambda": tune.loguniform(1e-4, 1e-2),
    #     "lr": tune.loguniform(1e-4, 1e-2),
    # }

    search_space = {
        "HB": tune.choice(valid_hb_pairs),
        "l1_lambda": tune.choice([0.00597965]),
        "lr": tune.choice([2.5011e-05]),
    }

    # scheduler_asha = tune.schedulers.ASHAScheduler(
    #     max_t=num_epochs,
    #     grace_period=1,
    #     reduction_factor=2,
    # )

    # os.environ["RAY_CHDIR_TO_TRIAL_DIR"] = "0" # Allows relative paths, but trials are not isolated

    trainable_with_resources = with_resources(
        train_model,
        {"cpu": 4, "gpu": 1}  # Adjust based on your available resources
    )

    tuner = Tuner(
        trainable=trainable_with_resources,
        param_space=search_space,
        tune_config=tune.TuneConfig(
            metric="val_loss",
            mode="min",
            num_samples=1, # Number of hyperparameter sets to try
            max_concurrent_trials=1, # Number of trials to run concurrently
            # scheduler=scheduler_asha,
        ),
        run_config=RunConfig(
            name="hyperparameter_search",
            storage_path=str(Path("./results").resolve()),
        ),
    )
    results = tuner.fit()
    best_result = results.get_best_result(metric="val_loss", mode="min")
    print("Best Hyperparameters Found:")
    print(best_result.config)
    return results

# # Run Hyperparameter Search
# if __name__ == "__main__":
#     tune_hyperparameters()

In [9]:
import multiprocessing
multiprocessing.set_start_method("spawn", force=True)


In [10]:
results = tune_hyperparameters()

(train_model pid=376) /opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
(train_model pid=376) /opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
(train_model pid=376)   self.pid = os.fork()


Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:05<00:00,  0.35it/s]
                                                                           
Epoch 0: 100%|██████████| 508/508 [27:51<00:00,  0.30it/s, v_num=0]
(train_model pid=376) 
Validation: |          | 0/? [00:00<?, ?it/s]
(train_model pid=376) 
Validation DataLoader 0:   0%|          | 0/8 [00:00<?, ?it/s]
(train_model pid=376) 
Validation DataLoader 0:  12%|█▎        | 1/8 [00:00<00:02,  2.33it/s]
(train_model pid=376) 
Validation DataLoader 0:  25%|██▌       | 2/8 [00:05<00:16,  0.36it/s]
(train_model pid=376) 
Validation DataLoader 0:  38%|███▊      | 3/8 [00:10<00:17,  0.28it/s]
(train_model pid=376) 
Validation DataLoader 0:  50%|█████     | 4/8 [00:16<00:16,  0.25it/s]
(train_model pid=376) 
Validation DataLoader 0:  62%|██████▎   | 5/8 [00:25<00:15,  0.19it/s]
(train_model pid=376) 
Validation DataLoader 0:  75%|███████▌  | 6/8 [00:30<00:10,  0.19it/s]
(train_mod

(train_model pid=376) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/kaggle/working/results/hyperparameter_search/train_model_b91fe_00000_0_HB=hidden_dim_65536_batch_size_2048,l1_lambda=0.0060,lr=0.0000_2024-12-04_16-56-42/checkpoint_000000)


(train_model pid=376) 
Epoch 0: 100%|██████████| 508/508 [29:21<00:00,  0.29it/s, v_num=0]


2024-12-04 17:27:15,949	WARNING util.py:201 -- The `on_step_begin` operation took 3.385 s, which may be a performance bottleneck.


...

(train_model pid=376) 
Epoch 3: 100%|██████████| 508/508 [27:20<00:00,  0.31it/s, v_num=0]
(train_model pid=376) 
Validation: |          | 0/? [00:00<?, ?it/s]
(train_model pid=376) 
Validation DataLoader 0:   0%|          | 0/8 [00:00<?, ?it/s]
(train_model pid=376) 
Validation DataLoader 0:  12%|█▎        | 1/8 [00:00<00:02,  2.37it/s]
(train_model pid=376) 
Validation DataLoader 0:  25%|██▌       | 2/8 [00:05<00:16,  0.36it/s]
(train_model pid=376) 
Validation DataLoader 0:  38%|███▊      | 3/8 [00:10<00:17,  0.28it/s]
(train_model pid=376) 
Validation DataLoader 0:  50%|█████     | 4/8 [00:15<00:15,  0.25it/s]
(train_model pid=376) 
Validation DataLoader 0:  62%|██████▎   | 5/8 [00:24<00:14,  0.20it/s]
(train_model pid=376) 
Validation DataLoader 0:  75%|███████▌  | 6/8 [00:30<00:10,  0.20it/s]
(train_model pid=376) 
Validation DataLoader 0:  88%|████████▊ | 7/8 [00:35<00:05,  0.20it/s]
(train_model pid=376) 
Validation DataLoader 0: 100%|██████████| 8/8 [00:40<00:00,  0.20it/s]


(train_model pid=376) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/kaggle/working/results/hyperparameter_search/train_model_b91fe_00000_0_HB=hidden_dim_65536_batch_size_2048,l1_lambda=0.0060,lr=0.0000_2024-12-04_16-56-42/checkpoint_000003)


(train_model pid=376) 
Epoch 3: 100%|██████████| 508/508 [29:23<00:00,  0.29it/s, v_num=0]


2024-12-04 18:55:50,094	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/kaggle/working/results/hyperparameter_search' in 0.0062s.
2024-12-04 18:55:50,099	INFO tune.py:1041 -- Total run time: 7153.70 seconds (7147.47 seconds for the tuning loop).


Best Hyperparameters Found:
{'HB': {'hidden_dim': 65536, 'batch_size': 2048}, 'l1_lambda': 0.00597965, 'lr': 2.5011e-05}
